Fixing missing blocks is a known issue between one census and another

In [ ]:
%matplotlib inline

# Assuming we are in the notebooks directory, we need to move one up:
%cd ..

from __future__ import division

import os
import us
import requests

import pandas as pd
import geopandas as gpd

from secret import *
from default_values import *

In [ ]:
state = u'arizona'
state_fips = us.states.lookup(state).fips
state_fips

In [ ]:
# try:
if True:
    print 'Attempting to load predicted population by block for:', state
#     df = gpd.read_file(blocks_geojson_filename)
# except IOError:
    print '\tNot found on disk...'
    print '\tDownloading and processing blocks and block groups for:', state
    # Open 2010 blocks for state (the population):
    print '\tOpening 2010 blocks shapefile for:', state
    blocks_2010_shapefile_name = blocks_2010_shapefile_dir + 'tabblock2010_' +\
        state_fips + '_pophu.shp'
    blocks_df = gpd.read_file(blocks_2010_shapefile_name)
    blocks_df['BLOCKID10'] = blocks_df['BLOCKID10'].apply(str)
    def get_group_GEOID(blockGEOID):
        return blockGEOID[0:-3]
    blocks_df['Block Group GEOID'] = blocks_df['BLOCKID10'].apply(get_group_GEOID)
    assert (blocks_df['Block Group GEOID'].apply(len)).unique() == [12]
    # The filename for the whole state with block groups (without population):
    print '\tOpening ACS5 block groups shapefile for:', state
    block_groups_shapefile_name = acs5_shapefile_dir + 'tl_' + year + '_' +\
        state_fips + '_bg.shp'
    block_groups_df = gpd.read_file(block_groups_shapefile_name)
    # We will add to this the population per block group from the API.
    # Check GEOIDs are of length 12 because they have to be by definition,
    # see: https://www.census.gov/geo/reference/geoidentifiers.html
    assert (block_groups_df['GEOID'].apply(len)).unique() == [12]
  
    
    
    print set(block_groups_df['GEOID'].unique()).symmetric_difference(set(
        blocks_df['Block Group GEOID'].unique()))
        # ['060378002043', '060379304011', u'060371370002', u'060371370001']

In [ ]:
diff = list(set(block_groups_df['GEOID'].unique()).symmetric_difference(set(
         blocks_df['Block Group GEOID'].unique())))
len(diff)

In [ ]:
for d in diff:
    if d in blocks_df['Block Group GEOID'].values:
        print d

In [ ]:
#These are the block groups that get lost:
for i, d in enumerate(diff):
#     print i
#     d = str(d)
#     print block_groups_df[block_groups_df['GEOID'].astype(str).str.contains(d)]
    try:
        lost_block_groups = pd.concat([lost_block_groups,
                                      block_groups_df[block_groups_df['GEOID'].astype(str).str.contains(d)]])
    except NameError:
        lost_block_groups = block_groups_df[block_groups_df['GEOID'].astype(str).str.contains(d)]
lost_block_groups.shape

In [ ]:
lost_block_groups

In [ ]:
    print len(set(block_groups_df['GEOID'].unique()).difference(set(
        blocks_df['Block Group GEOID'].unique())))


In [ ]:
    print len(set(
         blocks_df['Block Group GEOID'].unique()).difference(set(block_groups_df['GEOID'].unique())))

In [ ]:
    print blocks_df[blocks_df['Block Group GEOID'] == '040194705002']['POP10']


In [ ]:
    print blocks_df[blocks_df['Block Group GEOID'] == '060371370001']['POP10']

In [ ]:
        # Code to get the total estimated population from the census files:
        api_population = 'B01003_001E'  # http://api.census.gov/data/2015/acs5/variables.html
        # NB: this is not the same code in the shapefiles for each block from the 2010
        # census, which instead uses 'POP10'.
        # Get all the counties' IDs so we can d/l each county's population per
        # block group:
        def block_group_to_county_geoid(geoid):
            return geoid[2:5]
        block_groups_df['County'] = block_groups_df['GEOID'].apply(block_group_to_county_geoid)
        block_groups_for_state = acs5_population_dir + state + '.json'
        try:
            print '\tAttempting to load ACS5 population for block groups by county for:', state
            with open(block_groups_for_state, 'r') as infile:
                population_df = pd.read_json(infile)
        except IOError:
            print '\t\tNot found on disk...'
            counties = block_groups_df['County'].unique()
            for county in counties:
                print '\t\tDownloading block groups for county:', county
                url = 'http://api.census.gov/data/' + year + '/acs5?get=NAME,' +\
                    api_population + '&for=block+group:*&in=state:' + \
                    state_fips + '+county:' + county + '&key=' + apikey
                # Make a get request to get the population of each county per block
                # group:
                response = requests.get(url)
                try:
                    data = response.json()
                except ValueError:  # includes simplejson.decoder.JSONDecodeError
                    # This is what happens when the server times out etc.
                    print 'Decoding JSON has failed. Server response:',\
                        response.status_code
                    exit()
                # Merge the df to get one huge df for the whole state which contains
                # all the group blocks and their populations:
                columns = data.pop(0) # columns as list and leaves data intact
                try:
                    # Try appending to previous population_df:
                    population_df = population_df.append(
                        pd.DataFrame(data, columns=columns),
                        ignore_index=True)
                except NameError:
                    # There is no previous, so create it:
                    population_df = pd.DataFrame(data, columns=columns)
            population_df.to_json(block_groups_for_state)
        print '\tDone!'

In [ ]:
    def create_geoid(row):
        # GEOID Structure is defined as STATE + COUNTY + TRACT + BLOCK GROUP =
        # 2 + 3 + 6 + 1 = 12
        # see: https://www.census.gov/geo/reference/geoidentifiers.html
        STATE = str(row['state']).zfill(2)  # zero padding to conform to GEOID
        COUNTY = str(row['county']).zfill(3)
        TRACT = str(row['tract']).zfill(6)
        BLOCK_GROUP = str(row['block group'])
        assert len(STATE + COUNTY + TRACT + BLOCK_GROUP) == 12
        return STATE + COUNTY + TRACT + BLOCK_GROUP
    population_df['GEOID'] = population_df.apply(create_geoid, axis=1)
    
    

In [ ]:
block_groups_df.head()

In [ ]:
    block_groups_df = block_groups_df.merge(population_df, on='GEOID')
    assert block_groups_df['GEOID'].shape == population_df['GEOID'].shape
#     del population_df
    # Get rid of other columns, we are just interested in below for each block
    # group:
    block_groups_df = block_groups_df.loc[:,['GEOID',api_population]]
    # We need to calculate the proportion of the populalion of 2010 block group
    # that a constituent 2010 block has:
    # a. get the total population for each block group from 2010 and associate
    # each 2010 block with its total:
    blocks_df['POP10'] = pd.to_numeric(blocks_df['POP10'])
    blocks_df = blocks_df.join(blocks_df.groupby('Block Group GEOID')[
                               'POP10'].sum(), on='Block Group GEOID',
                                rsuffix=' for 2010 Block Group')
    blocks_df.rename(columns={'POP10 for 2010 Block Group':\
                              'Population for 2010 Block Group'}, inplace=True)

In [ ]:
blocks_df.head()

In [ ]:

# b. get the ratio of block population to respective block group
# population:
def calculate_ratio(row):
    try:
        return row['POP10'] / row['Population for 2010 Block Group']
    except:
        return row['Population for 2010 Block Group']
blocks_df['Population Ratio'] = blocks_df.apply(calculate_ratio, axis=1)
# c. get the population for the 2015 ACS5 block groups and associate them
# with their respective blocks:
blocks_df = pd.merge(blocks_df, block_groups_df,
                     left_on = 'Block Group GEOID',
                     right_on = 'GEOID')
blocks_df.rename(columns={api_population:\
                    'Population for ' + year + ' Block Group'},
                 inplace=True)
assert (blocks_df['GEOID'] == blocks_df['Block Group GEOID']).all()
del blocks_df['GEOID']
blocks_df['Population for ' + year + ' Block Group'] =\
    pd.to_numeric(blocks_df['Population for ' + year + ' Block Group'])
assert block_groups_df[api_population].sum() > blocks_df['POP10'].sum()
# assert blocks_df['Population for 2010 Block Group'].sum() <\
    # blocks_df['Population for ' + year + ' Block Group'].sum()
#     del block_groups_df # we do not need this anymore
# d. calculate the predicted population for the block based on the above:
blocks_df[population] = blocks_df['Population Ratio'] *\
    blocks_df['Population for ' + year + ' Block Group']
# Calculate the centroids for each block required for clustering:
def get_x(p): return p.x
def get_y(p): return p.y
blocks_df['Centroid Longitude'] = blocks_df['geometry'].centroid.apply(get_x)
blocks_df['Centroid Latitude'] = blocks_df['geometry'].centroid.apply(get_y)
# Find which congressional district each block group for this state belongs
#  to. This is done using the block assignments files downloaded from:
#  https://www.census.gov/geo/maps-data/data/baf.html
# print '\tCalculating congressional districts for:', state
block_assignments_df = pd.read_csv(block_assignments_dir +\
    'National_CD115.txt', dtype={"BLOCKID": str, "CD115": str})
block_assignments_df.rename(columns={'BLOCKID': 'GEOID'}, inplace=True)
blocks_df.rename(columns={'BLOCKID10': 'GEOID'}, inplace=True)
blocks_df = blocks_df.merge(block_assignments_df, how='left',
    indicator=True)
assert blocks_df['_merge'].unique() == ['both']
del blocks_df['_merge']
del block_assignments_df
# We are done now, check that columns are what we expect before we remove
# useless ones:
assert list(blocks_df) ==\
    [u'BLOCKCE', 'GEOID', u'COUNTYFP10', u'HOUSING10', u'PARTFLG', u'POP10',
     u'STATEFP10', u'TRACTCE10', 'geometry', 'Block Group GEOID',
     'Population for 2010 Block Group', 'Population Ratio',
     'Population for 2015 Block Group', 'Predicted 2015 Population',
     'Centroid Longitude', 'Centroid Latitude', 'CD115']


In [ ]:
blocks_df.head()

In [ ]:
lost_block_groups.head()

In [ ]:

    # Get rid of colums we do not need:
blocks_df = blocks_df.loc[:,['GEOID', 'CD115', 'geometry',
                                 'Predicted 2015 Population',
                                 'Centroid Longitude', 'Centroid Latitude']]

In [ ]:
blocks_df.head()

In [ ]:

# Give columns better names:
blocks_df.rename(columns={'CD115': 'Congressional District',
                          'Centroid Longitude': 'Centroid Longitude',
                          'Centroid Latitude': 'Centroid Latitude'},
                  inplace=True)
blocks_df['Congressional District'] = blocks_df['Congressional District'].apply(str)
blocks_df['Centroid Longitude'] = blocks_df['Centroid Longitude'].apply(float)
blocks_df['Centroid Latitude'] = blocks_df['Centroid Latitude'].apply(float)
blocks_df['Predicted 2015 Population'] = blocks_df['Predicted 2015 Population'].apply(float)
# We are finally done, save the file!
print 'Saving file...'
blocks_df = gpd.GeoDataFrame(blocks_df)
with open(blocks_geojson_filename, 'w') as outfile:
    outfile.write(blocks_df.to_json())
# In the rest of this code the block-wise dataframe with predicted
# populations is called just df:
df = blocks_df
del blocks_df
print 'Done!'

In [ ]:
# del lost_block_groups['ALAND']
# del lost_block_groups['AWATER']
# del lost_block_groups['BLKGRPCE']

population_df.head()
lost_block_groups = lost_block_groups.merge(population_df, on='GEOID')

In [ ]:
lost_block_groups.head()
lost_block_groups['Centroid Longitude'] = lost_block_groups['geometry'].centroid.apply(get_x)
    
lost_block_groups['Centroid Latitude'] = lost_block_groups['geometry'].centroid.apply(get_y)

In [ ]:
lost_block_groups.rename(columns={'B01003_001E': 'Predicted 2015 Population'}, inplace=True)

lost_block_groups = lost_block_groups[['GEOID', 'geometry', 'Predicted 2015 Population', 'Centroid Longitude', 'Centroid Latitude']]
lost_block_groups.head()


In [ ]:
df.tail()

In [ ]:
block_groups_df.head()

In [ ]:
usa = gpd.read_file(usa_shapefile_path)
cong_dist = usa[usa['STATEFP'].apply(
    int) == int(state_fips)]
del usa

In [ ]:
cong_dist.head()

In [ ]:
missing_bg_cong_dist = pd.DataFrame(columns=[['GEOID', 'Congressional District']])
missing_bg_cong_dist.head()

In [ ]:
geoid = []
cd = []
for bg_index, bg_row in lost_block_groups.iterrows():
    for dist_index, dist_row in cong_dist.iterrows():
        bg_area =  bg_row['geometry'].area
        intersection_area = bg_row['geometry'].intersection(dist_row['geometry']).area
        if bg_area - intersection_area < 1e-10:
#             print bg_index, dist_row['CD115FP']
#             print intersection_area, bg_area
            geoid.append(bg_row['GEOID'])
            cd.append(dist_row['CD115FP'])
        
geoid, cd

In [ ]:
missing_bg_cong_dist['GEOID'] = geoid
missing_bg_cong_dist['Congressional District'] = cd
missing_bg_cong_dist.head()

In [ ]:
lost_block_groups = lost_block_groups.merge(missing_bg_cong_dist, on = 'GEOID')
# del lost_block_groups['Congressional District_x']
# del lost_block_groups['Congressional District_y']

In [ ]:
lost_block_groups.tail()


In [ ]:
df.head()

In [ ]:
df.shape
df = df.append(lost_block_groups, ignore_index=True)
df.shape

In [ ]:
df.dissolve(by='Congressional District').plot()

In [ ]:
with open(prediction_geojson_dir+'AZ.geojson', 'w') as outfile:
    outfile.write(df.to_json())